<a href="https://www.kaggle.com/code/tanakataro73/kaggle-book-morohashi-08-mlb-engagement?scriptVersionId=172765098" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# Python3.10 だと mlb モジュールが使えないため、3.7にする
# 参考: # 参考: https://www.kaggle.com/discussions/general/173536
# Internet on でないとインストールができないが、 Internet off　でないとノートを提出できない
# 詰んでいる
!conda create -n newCondaEnvironment -c cctbx202208 -y
!source /opt/conda/bin/activate newCondaEnvironment && conda install -c cctbx202208 python=3.7 -y

Retrieving notices: ...working... done
Channels:
 - cctbx202208
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/newCondaEnvironment



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate newCondaEnvironment
#
# To deactivate an active environment, use
#
#     $ conda deactivate

Channels:
 - cctbx202208
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/newCondaEnvironment

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  cctbx202208
    _openmp_mutex-4.5          |            2_gnu  

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import pickle
import os
import datetime as dt

import matplotlib.pyplot as plt

import lightgbm as lgb

from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter("ignore")

# https://stackoverflow.com/a/8885688
pd.options.display.float_format = "{:10.4f}".format

In [ ]:
train = pd.read_csv("/kaggle/input/mlb-player-digital-engagement-forecasting/train_updated.csv")
print(train.shape)
train.head()

In [ ]:
# 試行錯誤の速度向上のため20200401以降にしぼる
train = train.loc[train["date"] >= 20200401, :].reset_index(drop=True)
print(train.shape)

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

def extract_data(
    input_df,
    col="events",
    show=False
):
    output_df = pd.DataFrame()
    
    # なぜrangeでなくarangeを使うのか謎だったが右が参考になる https://stackoverflow.com/a/75533644
    for i in np.arange(len(input_df)):
        if show:
            print("\r{}/{}".format(i+1, len(input_df), end=""))
        try:
            output_df = pd.concat(
                [
                    output_df,
                    unpack_json(input_df[col].iloc[i]),
                ],
                axis=0,
                ignore_index=True,
            )
        except:
            pass
    
    if show:
        print("")
        print(output_df.shape)
        display(output_df.head())

    return output_df

In [ ]:
df_engagement = extract_data(train, col="nextDayPlayerEngagement", show=True)

In [ ]:
df_engagement["date_playerId"] = df_engagement["engagementMetricsDate"].str.replace("-", "") + "_" + df_engagement["playerId"].astype(str)
df_engagement.head()

In [ ]:
# 推論実施日＝推論対象日の前日
df_engagement["date"] = pd.to_datetime(df_engagement["engagementMetricsDate"], format="%Y-%m-%d") + dt.timedelta(days=-1)
# 推論実施日から曜日と年月の特徴量作成
df_engagement["dayofweek"] = df_engagement["date"].dt.dayofweek
df_engagement["yearmonth"] = df_engagement["date"].astype(str).apply(lambda x: x[:7])
df_engagement.head()

In [ ]:
# players.csv
df_players = pd.read_csv("/kaggle/input/mlb-player-digital-engagement-forecasting/players.csv")
print(df_players.shape)
print(df_players["playerId"].agg("nunique"))
df_players.head()

In [ ]:
df_players["playerForTestSetAndFuturePreds"] = np.where(df_players["playerForTestSetAndFuturePreds"] == True, 1, 0)
print(df_players["playerForTestSetAndFuturePreds"].sum())
print(df_players["playerForTestSetAndFuturePreds"].mean())

In [ ]:
df_engagement.head()

In [ ]:
df_players.head()

In [ ]:
# df_engagement に df_players を LEFT JOIN
df_train = pd.merge(df_engagement, df_players, on=["playerId"], how="left")
print(df_train.shape)

In [ ]:
x_train = df_train[[
    "playerId",
    "dayofweek",
    "birthCity",
    "birthStateProvince",
    "birthCountry",
    "heightInches",
    "weight",
    "primaryPositionCode",
    "primaryPositionName",
    "playerForTestSetAndFuturePreds",
]]

y_train = df_train[[
    "target1",
    "target2",
    "target3",
    "target4",
]]

id_train = df_train[[
    "engagementMetricsDate",
    "playerId",
    "date_playerId",
    "date",
    "yearmonth",
    "playerForTestSetAndFuturePreds",
]]

print(x_train.shape, y_train.shape, id_train.shape)
x_train.head()

In [ ]:
# lightgbmで扱うためcategory型に
for col in [
    "playerId",
    "dayofweek",
    "birthCity",
    "birthStateProvince",
    "birthCountry",
    "primaryPositionCode",
    "primaryPositionName",
]:
    x_train[col] = x_train[col].astype("category")

In [ ]:
df_train.sort_values("date").tail()

# 8.3.4 バリデーション設計

## 説明変数として使って良いデータはなにか
- デジタルエンゲージメント（target1, target2, ...）は2021/7/31ぶんまで利用できる。それ以降の日付のぶんは利用できない
    - つまり説明変数として使う場合、8/1は前日より過去の値、8/2なら2日前よりも過去の値、・・・、8/31なら31日前より過去の値を利用できる
    - 31パターンのモデルを作成する手もあるが管理が大変なので、ベースラインでは1モデルとし、「先月のデジタルエンゲージメントなら利用可能」とする

## 古いデータは学習に使うべきか
- 著者の環境では期間を増やすほど精度が改善したとのことだが、ベースラインでは「直近1年」(=検証データ含め13ヶ月)とする

## 学習用データセットから検証データをどう作るか
- ベースラインでは検証データ期間を「2021/5」「2021/6」「2021/7」のパターンにわける。パターンごとに、直近12ヶ月を学習データとする

In [ ]:
# 学習データと検証データの期間のリスト
list_cv_month = [
    [
        [
            "2020-05",
            "2020-06",
            "2020-07",
            "2020-08",
            "2020-09",
            "2020-10",
            "2020-11",
            "2020-12",
            "2021-01",
            "2021-02",
            "2021-03",
            "2021-04",
        ],
        [
            "2021-05",
        ],
    ],
    [
        [
            "2020-06",
            "2020-07",
            "2020-08",
            "2020-09",
            "2020-10",
            "2020-11",
            "2020-12",
            "2021-01",
            "2021-02",
            "2021-03",
            "2021-04",
            "2021-05",
        ],
        [
            "2021-06",
        ],
    ],
    [
        [
            "2020-07",
            "2020-08",
            "2020-09",
            "2020-10",
            "2020-11",
            "2020-12",
            "2021-01",
            "2021-02",
            "2021-03",
            "2021-04",
            "2021-05",
            "2021-06",
        ],
        [
            "2021-07",        
        ],
    ]
]

In [ ]:
def train_lgb(
    input_x,
    input_y,
    input_id,
    params: dict,
    list_nfold=[0, 1, 2],
    mode_train="train",
):
    # 推論値を格納する変数
    df_valid_pred = pd.DataFrame()
    # 評価値を入れる変数
    metrics = []
    # 重要度を格納する変数
    df_imp = pd.DataFrame()
    
    # corss validation 用のインデクス作成
    cv = []
    for month_tr, month_va in list_cv_month:
        _cv = [
            input_id.index[input_id["yearmonth"].isin(month_tr)],
            input_id.index[input_id["yearmonth"].isin(month_va) & (input_id["playerForTestSetAndFuturePreds"] == 1)],
        ]
        cv.append(_cv)

    # fold * target 毎にモデル学習
    for nfold in list_nfold:
        for i, target in enumerate(["target1", "target2", "target3", "target4"]):
            print("-"*20, target, ", fold:", nfold, "-"*20)
            
            idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
            x_tr, y_tr, id_tr = input_x.loc[idx_tr, :], input_y.loc[idx_tr, target], input_id.loc[idx_tr, :]
            x_va, y_va, id_va = input_x.loc[idx_va, :], input_y.loc[idx_va, target], input_id.loc[idx_va, :]
            print(x_tr.shape, y_tr.shape, id_tr.shape)
            print(x_va.shape, y_va.shape, id_va.shape)
            
            # 保存するモデルのファイル名
            filepath = "model_lgb_{}_fold{}.h5".format(target, nfold)
            
            if mode_train == "train":
                print("training start.")
                model = lgb.LGBMRegressor(**params)
                model.fit(
                    x_tr,
                    y_tr,
                    eval_set=[(x_tr, y_tr), (x_va, y_va)],
                    callbacks=[lgb.early_stopping(stopping_rounds=50),],
                )
                with open(filepath, "wb") as f:
                    pickle.dump(model, f, protocol=4)
            else:
                print("model load.")
                with open(filepath, "rb") as f:
                    model = pickle.load(f)
                print("Done.")
            
            # validの推論値取得
            y_va_pred = model.predict(x_va)
            tmp_pred = pd.concat(
                [
                    id_va,
                    pd.DataFrame({
                        "target": target,
                        "nfold": nfold,
                        "true": y_va,
                        "pred": y_va_pred,
                    })
                ],
                axis=1,
            )
            df_valid_pred = pd.concat([df_valid_pred, tmp_pred], axis=0, ignore_index=True)
            
            # 評価値の算出
            metric_va = mean_absolute_error(y_va, y_va_pred)
            metrics.append([target, nfold, metric_va])
            
            # 重要度
            tmp_imp = pd.DataFrame({
                "col": x_tr.columns,
                "imp": model.feature_importances_,
                "target": target,
                "nfold": nfold,
            })
            df_imp = pd.concat(
                [
                    df_imp,
                    tmp_imp,
                ],
                axis=0,
                ignore_index=True
            )
            
    print("-"*10, "result", "-"*10)
    # 評価値
    df_metrics = pd.DataFrame(metrics, columns=["target", "nfold", "mae"])
    print("MCMAE: {:.4f}".format(df_metrics["mae"].mean()))
    
    # validの推論値
    df_valid_pred_all = pd.pivot_table(
        df_valid_pred,
        # プレイヤー * 日付 ごとに
        index=[
            "engagementMetricsDate",
            "playerId",
            "date_playerId",
            "date",
            "yearmonth",
            "playerForTestSetAndFuturePreds",
        ],
        # target * fold ごとの
        columns=[
            "target",
            "nfold",
        ],
        # 真の値と予測値を算出
        values=[
            "true",
            "pred"
        ],
        # なぜ sum を使うのか?
        aggfunc=np.sum,
    )
    df_valid_pred_all.columns = [
        "{}_fold{}_{}".format(j, k, i) for i, j, k in df_valid_pred_all.columns
    ]
    df_valid_pred_all = df_valid_pred_all.reset_index(drop=False)
    
    return df_valid_pred_all, df_metrics, df_imp

In [ ]:
# モデル学習
params = {
    "boosting_type": "gbdt",
    "objective": "regression_l1",
    "metric": "mean_absolute_error",
    "learning_rate": 0.05,
    "num_leaves": 32,
    "subsample": 0.7,
    "subsample_freq": 1,
    "feature_fraction": 0.8,
    "min_data_in_leaf": 50,
    "min_sum_hessian_in_leaf": 50,
    "n_estimators": 1000,
    "random_state": 123,
    "importance_type": "gain",
}

df_valid_pred, df_metrics, df_imp = train_lgb(
    x_train,
    y_train,
    id_train,
    params,
)

In [ ]:
print("MCMAE: {:.4f}".format(df_metrics["mae"].mean()))
display(
    pd.pivot_table(
        df_metrics,
        index="nfold",
        columns="target",
        values="mae",
        aggfunc=np.mean,
        margins=True,
    )
)

In [ ]:
df_imp.groupby(["col"])["imp"].agg(["mean", "std"]).sort_values("mean", ascending=False)

# 8.3.6 モデル推論

In [ ]:
# ModuleNotFoundError: No module named 'mlb.competition' が出る -> Python3.7でないとだめ
# 参考： https://www.kaggle.com/competitions/mlb-player-digital-engagement-forecasting/discussion/248839

# 推論時に受け取るデータのフォーマット確認
# import mlb
# env = mlb.make_env() # initialize the environment
# iter_test = env.iter_test() # iterator which loops over each date in test set

# for (test_df, prediction_df) in iter_test:
#     display(test_df.head())
#     display(prediction_df.head())
#     break

In [ ]:
# 3.10
!python --version